# Bibliotecas:

## Objetivo:

_Remontar a tabela do brasileirão com o parsel._

_Spoiler: ela é realmente mais prática que a bs4._

In [1]:
from parsel import Selector
import requests
import pandas as pd
from datetime import date

In [2]:
busca = requests.get("https://www.lance.com.br/tabela/brasileirao")

In [3]:
html = busca.text

In [4]:
sel = Selector(text=html)

In [5]:
def parse_data(seletor):
    rk = []
    for item in seletor:
        p = item.css('::text').getall()[0]
        rk.append(p)
    return rk

# Classificação do times:

In [6]:
ranking = sel.css('span.styles_teamClassification__fVPxP')

In [7]:
rk = []
for item in ranking:
    p = item.css('::text').getall()[0]
    rk.append(int(p))

In [8]:
rk

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

# Nomes do Clubes:

In [9]:
clubes = sel.css('span.styles_teamName__QJ4sa')

In [10]:
parse_data(clubes)

['Palmeiras',
 'Grêmio',
 'Atlético-MG',
 'Flamengo',
 'Botafogo',
 'Red Bull Bragantino',
 'Fluminense',
 'Athletico-PR',
 'Internacional',
 'Fortaleza',
 'São Paulo',
 'Cuiabá',
 'Corinthians',
 'Cruzeiro',
 'Vasco',
 'Bahia',
 'Santos',
 'Goiás',
 'Coritiba',
 'América-MG']

# Pontos:

In [11]:
pontos = sel.css("div.styles_tableContentGrid__Nyval")

In [12]:
pontos.css('div.styles_td__avyCB::text').get()

'70'

In [13]:
x = pontos.css('div.styles_td__avyCB::text').getall()

In [14]:
def chunks(lista, n):
    inicio = 0
    for i in range(n):
        final = inicio + len(lista[i::n])
        yield lista[inicio:final]
        inicio = final

In [15]:
dis_pontos = list(chunks(x, 20))


In [16]:
dt_pontos = {}
idx = 1
for i in dis_pontos:
    
    dt_pontos[f'{idx}'] = i
    idx+=1
    

# Montagem da Tabela:

In [17]:
brasileiro = pd.DataFrame(dt_pontos)
brasileiro = brasileiro.T

In [18]:
brasileiro.insert(0, 'Clubes', parse_data(clubes))

In [19]:
brasileiro = brasileiro.rename(columns={0: 'Pontos', 1:"Partidas Jogadas", 2:"Vitorias", 3:"Empates", 4:"Derrotas", 5:"Gols Marcados", 6:"Gols Contra", 7:"Saldo de Gols", 8:"%"})

In [20]:
display(brasileiro)

,Clubes,Pontos,Partidas Jogadas,Vitorias,Empates,Derrotas,Gols Marcados,Gols Contra,Saldo de Gols,%
1,Palmeiras,70,38,20,10,8,64,33,31,61
2,Grêmio,68,38,21,5,12,63,56,7,60
3,Atlético-MG,66,38,19,9,10,52,32,20,58
4,Flamengo,66,38,19,9,10,56,42,14,58
5,Botafogo,64,38,18,10,10,58,37,21,56
6,Red Bull Bragantino,62,38,17,11,10,49,35,14,54
7,Fluminense,56,38,16,8,14,51,47,4,49
8,Athletico-PR,56,38,14,14,10,51,43,8,49
9,Internacional,55,38,15,10,13,46,45,1,48
10,Fortaleza,54,38,15,9,14,45,44,1,47


In [21]:
brasileiro.head(5)

,Clubes,Pontos,Partidas Jogadas,Vitorias,Empates,Derrotas,Gols Marcados,Gols Contra,Saldo de Gols,%
1,Palmeiras,70,38,20,10,8,64,33,31,61
2,Grêmio,68,38,21,5,12,63,56,7,60
3,Atlético-MG,66,38,19,9,10,52,32,20,58
4,Flamengo,66,38,19,9,10,56,42,14,58
5,Botafogo,64,38,18,10,10,58,37,21,56


In [22]:
file_name = f"{str(date.today())}_tabela_brasileiro.xlsx"
brasileiro.to_excel(file_name)